In [127]:
from datetime import datetime
from lxml import html
import requests
import json
import pandas as pd
import numpy as np
import re
import time

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', None)



__author__ = 'aituarov'


DB_LOADER_DIR = 'C:\\DEV\\output\\'
DATAFILES_DIR = 'C:\\DEV\\output\\'


json_file = 'C:\\DEV\\google_translate_v2.json'


header = {
    'Business name_': 'company',
    'Hydroelectric power plant_general': 'Hydro',
    'Hydroelectric power plant_Pumped': 'Pumped Hydro',
    'Thermal power plant_coal': 'coal',
    'Thermal power plant_ＬＮＧ': 'LNG',
    'Thermal power plant_oil': 'oil',
    'Thermal power plant_LPG': 'LPG',
    'Thermal power plant_Other gas': 'Gas other',
    'Thermal power plant_Cyanogen mixture': 'Bituminous mixture',
    'Thermal power plant_Other': 'Thermal other',
    'Nuclear power plant_': 'Nuclear',
    'New energy power plant_Wind force': 'Wind',
    'New energy power plant_sunshine': 'Solar',
    'New energy power plant_Geothermal': 'Geothermal',
    'New energy power plant_biomass':'Biomass',
    'New energy power plant_waste':'Waste',
    'Other_':'Other'   
}


def download_file():
    print("Start downloading file...")
    url = 'https://www.enecho.meti.go.jp/statistics/electric_power/ep002/xls/2019/2-1-2019.xlsx'
    response = requests.get(url)
    file_name = DATAFILES_DIR + 'owner_raw_2019.xlsx'
    with open(file_name, 'wb') as output:
        output.write(response.content)
        output.close()
    print("File downloaded")
    
    
def json_to_dictionary(str):
    with open(json_file) as j:
        dic = json.load(j)
        
    for d in dic:
        str = str.replace(d,dic[d])

    return str
    
    
def translate_file(file_name):
    myDict = {}
    length = 0
    
    print("Translating sheets of downloaded file...")
    data_table = pd.DataFrame()
    
    xl = pd.ExcelFile(file_name)
    needed_sheets_names = []
    for sh_name in xl.sheet_names:
        if re.search(r'\d{4}.\d+', sh_name):
            needed_sheets_names.append(sh_name)

    raw_data = xl.parse(needed_sheets_names)

    data = {}

    for key in raw_data.keys():
        data[key] = json_to_dictionary(raw_data[key])
        headers = data[key].iloc[0]
        for ind in range(len(headers)):
            if isinstance(headers[ind], float):
                headers[ind] = headers[ind-1]
    
        sub_headers = data[key].iloc[1].replace(np.nan, '')
        headers = headers + '_' + sub_headers
        data[key].columns = headers
        data[key] = data[key].rename(columns=lambda x: header[x] if x in header.keys() else np.nan)
        data[key] = data[key].loc[data[key]['company'].notnull(), data[key].columns.notnull()].reset_index(drop=True).head(-1).tail(-1)
        
        data[key] = pd.melt(data[key], id_vars=['company'], value_vars = list(data[key].columns)[1:], var_name='name', value_name='value')
        data[key] = change_format_df(data[key], key)
        data_table = data_table.append(data[key])
        print(key + " sheet translated")
        
        
    data_table.to_csv(DB_LOADER_DIR + "owner_full_2020_data.csv", sep=';',index=False)
    print("End.")


        
def change_format_df(data, key):
    data['name'] = data['company'].replace('/', '').replace('㈱Global New Energy Togo', 'Global New Energy Togo') + '_' + data['name']
    data['effective_date'] = ""
    data['start_date'] = datetime.strptime(key, '%Y.%m').strftime('%Y-%m-%d')
    data['end_date'] = ""
    data['value'] = data['value'].map(lambda x:round(x) if not (pd.isna(x) or isinstance(x, str)) else float('nan'))
    data['table'] = 'spot'
    data = data[["name","effective_date","start_date","end_date","value","table"]]
    
    return data

   
def main():
#     download_file()
#     time.sleep(3)
    translate_file(DATAFILES_DIR + 'owner_full_raw.xlsx')


if __name__ == '__main__':
    main()


Translating sheets of downloaded file...
2020.4 sheet translated
End.


In [55]:
# a = "津軽風力発電株式会社"
import json
import pandas as pd


a = pd.DataFrame([["コスモエコパワー株式会社", "㈱鹿児島風力発電研究所"], ["ふそう風力発電株式会社", "まほろば風力発電株式会社"], ["伊方エコ・パーク株式会社",
"八峰風力開発株式会社"]])
myDict = {}
# print(a)
# print(a.iloc[0])
# print(a.shape)
for row in range(a.shape[0]):
    for col in range(a.shape[1]):
        myDict[a.iloc[row, col]] = str(row) +"_" + str(col)
        
        
with open('C:\\DEV\\output\\sample.json', "w") as outfile:  
    json.dump(myDict, outfile) 
 
# print(myDict)
# print(len(myDict))
# for el in a.iloc[0]:
#     print('"' + str(el.encode('unicode_escape'))[2:-1].replace('\\', '\ ')x.replace(' \ ', '') + '"')

# unicode_a = '"' + str(a.encode('unicode_escape'))[2:-1].replace('\\', '\ ').replace(' \ ', '') + '"'

# print('a: ' + unicode_a)

# c = u"\u3231Global New Energy Togo"
# print('c: ' + c)

# print(a==c)

In [228]:
a = [
"コスモエコパワー株式会社",
"㈱鹿児島風力発電研究所",
"ふそう風力発電株式会社",
"まほろば風力発電株式会社",
"伊方エコ・パーク株式会社",
"八峰風力開発株式会社",
"印南風力発電株式会社",
"株式会社 葉山風力発電所",
"株式会社エムウインズ八竜",
"株式会社エヌエスウインドパワーひびき",
"株式会社ユーラスエナジー肝付",
"株式会社ユーラスエナジー野辺地",
"株式会社ユーラス江差風力",
"株式会社大川原ウインドファーム",
"江差ウインドパワー株式会社",
"江津ウィンドパワー株式会社",
"津軽パワー株式会社",
"男鹿風力発電株式会社",
"白馬ウインドファーム㈱",
"若美風力開発株式会社",
"頴娃風力発電株式会社"
]

b = [
"Cosmo Eco Power Co., Ltd.",
"Kagoshima Wind Power Research Institute Co., Ltd.",
"Fuso Wind Power Co., Ltd.",
"Mahoroba Wind Power Co., Ltd.",
"Ikata Eco Park Co., Ltd.",
"Happo Wind Development Co., Ltd.",
"Innan Wind Power Co., Ltd.",
"Hayama Wind Power Co., Ltd.",
"M Winds Hachiryu Co., Ltd.",
"NS Wind Power Hibiki Co., Ltd.",
"Eurus Energy Co., Ltd.",
"Eurus Energy Noheji Co., Ltd.",
"Eurus Esashi Wind Power Co., Ltd.",
"Okawara Wind Farm Co., Ltd.",
"Esashi Wind Power Co., Ltd.",
"Gotsu Wind Power Co., Ltd.",
"Tsugaru Power Co., Ltd.",
"Oga Wind Power Co., Ltd.",
"Hakuba Wind Farm Co., Ltd.",
"Wakami Wind Development Co., Ltd.",
"Ei Wind Power Co., Ltd."
]


dic = {}

for ind in range(len(a)):
#     print(el)
    unicode = (str(a[ind].encode('unicode_escape'))[2:-1].replace('\\', '\ ').replace(' \ ', ''))
#     print(unicode)
    dic[unicode] = b[ind]

    
print(dic)

{'\\u30b3\\u30b9\\u30e2\\u30a8\\u30b3\\u30d1\\u30ef\\u30fc\\u682a\\u5f0f\\u4f1a\\u793e': 'Cosmo Eco Power Co., Ltd.', '\\u3231\\u9e7f\\u5150\\u5cf6\\u98a8\\u529b\\u767a\\u96fb\\u7814\\u7a76\\u6240': 'Kagoshima Wind Power Research Institute Co., Ltd.', '\\u3075\\u305d\\u3046\\u98a8\\u529b\\u767a\\u96fb\\u682a\\u5f0f\\u4f1a\\u793e': 'Fuso Wind Power Co., Ltd.', '\\u307e\\u307b\\u308d\\u3070\\u98a8\\u529b\\u767a\\u96fb\\u682a\\u5f0f\\u4f1a\\u793e': 'Mahoroba Wind Power Co., Ltd.', '\\u4f0a\\u65b9\\u30a8\\u30b3\\u30fb\\u30d1\\u30fc\\u30af\\u682a\\u5f0f\\u4f1a\\u793e': 'Ikata Eco Park Co., Ltd.', '\\u516b\\u5cf0\\u98a8\\u529b\\u958b\\u767a\\u682a\\u5f0f\\u4f1a\\u793e': 'Happo Wind Development Co., Ltd.', '\\u5370\\u5357\\u98a8\\u529b\\u767a\\u96fb\\u682a\\u5f0f\\u4f1a\\u793e': 'Innan Wind Power Co., Ltd.', '\\u682a\\u5f0f\\u4f1a\\u793e\\ u8449\\u5c71\\u98a8\\u529b\\u767a\\u96fb\\u6240': 'Hayama Wind Power Co., Ltd.', '\\u682a\\u5f0f\\u4f1a\\u793e\\u30a8\\u30e0\\u30a6\\u30a4\\u30f3\\u30ba\\u

In [112]:
import pandas as pd
a = "㈱"
print('"' + str(a.encode('unicode_escape'))[2:-1].replace('\\', '\ ').replace(' \ ', '') + '"')

b = '㈱G'
print(b.encode('unicode_escape'))
c = "name"
print(c)
a = b.replace('㈱G', "_") + c
print(a)

new = pd.DataFrame(['㈱G'])
new[1] = new[0].replace('㈱', "_")
print(new)
# print(d)
# print(new)
# print(new)

"\u3231"
b'\\u3231G'
name
_name
    0   1
0  ㈱G  ㈱G


In [52]:
header = {
    'Company name_': 'company',
    'Hydroelectric power plant_general': 'Hydro',
    'Hydroelectric power plant_Pumping': 'Pumped Hydro',
    'Thermal power plant_coal': 'coal',
    'Thermal power plant_LNG': 'LNG',
    'Thermal power plant_oil': 'oil',
    'Thermal power plant_LPG': 'LPG',
    'Thermal power plant_Other gas': 'Gas other',
    'Thermal power plant_Bituminous mixture': 'Bituminous mixture',
    'Thermal power plant_Other': 'Thermal other',
    'Nuclear power plant_': 'Nuclear',
    'New energy power plant_Wind power': 'Wind',
    'New energy power plant_sunshine': 'Solar',
    'New energy power plant_Geothermal': 'Geothermal',
    'New energy power plant_biomass':'Biomass',
    'New energy power plant_waste':'Waste',
    'Other_':'Other'   
}
type(header.keys())

dict_keys

In [3]:
a = float('nan')
pd.isna(a)

NameError: name 'pd' is not defined

In [12]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.enecho.meti.go.jp/statistics/electric_power/ep002/results.html'

page = requests.get(url)

soup = BeautifulSoup(page.content, 'lxml')

s = soup.findAll("div", {"class": "box2"})

for link in s:
    print(link)

# print(s)
    


<div class="box2">
<div class="tileListCol1st h15em">
<h3>結果概要</h3>
<ul>
<li><a class="pdf" href="pdf/2019/0-2019.pdf">結果概要 (PDF形式:639KB)<span class="tag2">NEW</span></a></li>
</ul>
</div>
<div class="tileListCol2nd h15em">
<h3>発電所数・出力</h3>
<ul>
<li><a class="excel" href="xls/2019/1-1-2019.xlsx">1-(1) 電気事業者の発電所数、出力 (xlsx形式:2,524KB)<span class="tag2">NEW</span></a></li>
<li><a class="excel" href="xls/2019/1-2-2019.xlsx">1-(2) 都道府県別発電所数、出力 (xlsx形式:127KB)<span class="tag2">NEW</span></a></li>
</ul>
</div>
</div>
<div class="box2">
<div class="tileListCol1st h15em">
<h3>発電実績</h3>
<ul>
<li><a class="excel" href="xls/2019/2-1-2019.xlsx">2-(1) 発電実績 (xlsx形式:1,581KB)<span class="tag2">NEW</span></a></li>
<li><a class="excel" href="xls/2019/2-2-2019.xlsx">2-(2) 都道府県別発電実績 (xlsx形式:118KB)<span class="tag2">NEW</span></a></li>
</ul>
</div>
<div class="tileListCol2nd h15em">
<h3>需要実績</h3>
<ul>
<li><a class="excel" href="xls/2019/3-1-2019.xlsx">3-(1) 電力需要実績 (xlsx形式:776KB)<span class="tag2">NEW</span><

In [125]:
import pandas as pd
a = {'a': 'A', 'b': 'b'}

b = pd.DataFrame([['a', 'b'], ['c', 'd']])
for row in range(b.shape[0]):
    for col in range(b.shape[1]):
        if b.iloc[row, col] in a:
            print("OK")
        else:
            a[b.iloc[row, col]] = str(row) + "_" + str(col)
            print("No")
print(a)

print(1 == 1 and 2 == 2)
# print(b)

OK
OK
No
No
{'a': 'A', 'b': 'b', 'c': '1_0', 'd': '1_1'}
True


In [ ]:
from datetime import datetime
from lxml import html
import requests
import json
import pandas as pd
import numpy as np
import re
import time

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', None)



__author__ = 'aituarov'


DB_LOADER_DIR = 'C:\\DEV\\output\\'
DATAFILES_DIR = 'C:\\DEV\\output\\'


json_file = 'C:\\DEV\\google_translate.json'


header = {
    'Business name_': 'company',
    'Hydroelectric power plant_general': 'Hydro',
    'Hydroelectric power plant_Pumped': 'Pumped Hydro',
    'Thermal power plant_coal': 'coal',
    'Thermal power plant_ＬＮＧ': 'LNG',
    'Thermal power plant_oil': 'oil',
    'Thermal power plant_LPG': 'LPG',
    'Thermal power plant_Other gas': 'Gas other',
    'Thermal power plant_Cyanogen mixture': 'Bituminous mixture',
    'Thermal power plant_Other': 'Thermal other',
    'Nuclear power plant_': 'Nuclear',
    'New energy power plant_Wind force': 'Wind',
    'New energy power plant_sunshine': 'Solar',
    'New energy power plant_Geothermal': 'Geothermal',
    'New energy power plant_biomass':'Biomass',
    'New energy power plant_waste':'Waste',
    'Other_':'Other'   
}


def download_file():
    url = 'https://www.enecho.meti.go.jp/statistics/electric_power/ep002/'
    response = requests.get(url + 'results.html')
    print("Trying to find file from " + url + 'resulst.html')
    page = html.fromstring(response.text)
    
    files = page.xpath('//div[@class="tileListCol1st h15em"]/ul/li/a')
    for file in files:
        if file.text.startswith('2-(1)'):
            url = url + file.get('href')
    response = requests.get(url)
    print("Starting download file from " + url)
    file_name = DATAFILES_DIR + 'owner_full_raw.xlsx'
    with open(file_name, 'wb') as output:
        output.write(response.content)
        output.close()
    print("File downloaded")
    
    
def json_to_dictionary(str):
    with open(json_file) as j:
        dic = json.load(j)
        
    for d in dic:
        str = str.replace(d,dic[d])

    return str
    
    
def translate_file(file_name):
    myDict = {}
    length = 0
    
    print("Translating sheets of downloaded file...")
    data_table = pd.DataFrame()
    
    xl = pd.ExcelFile(file_name)
    needed_sheets_names = []
    for sh_name in xl.sheet_names:
        if re.search(r'\d{4}.\d+', sh_name):
            needed_sheets_names.append(sh_name)

    raw_data = xl.parse(needed_sheets_names)

    data = {}

    for key in raw_data.keys():
        data[key] = json_to_dictionary(raw_data[key])
        headers = data[key].iloc[0]
        for ind in range(len(headers)):
            if isinstance(headers[ind], float):
                headers[ind] = headers[ind-1]
    
        sub_headers = data[key].iloc[1].replace(np.nan, '')
        headers = headers + '_' + sub_headers
        data[key].columns = headers
        data[key] = data[key].rename(columns=lambda x: header[x] if x in header.keys() else np.nan)
        data[key] = data[key].loc[data[key]['company'].notnull(), data[key].columns.notnull()].reset_index(drop=True).head(-1).tail(-1)
        
        data[key] = pd.melt(data[key], id_vars=['company'], value_vars = list(data[key].columns)[1:], var_name='name', value_name='value')
        data[key] = change_format_df(data[key], key)
        print(data[key])
        data_table = data_table.append(data[key])
        print(key + " sheet downloaded")
        
        
    data_table.to_csv(DB_LOADER_DIR + "owner_full_data.csv", sep=';',index=False)
    print("End.")


        
def change_format_df(data, key):
    data['name'] = data['company'].replace('/', '').replace('㈱Global New Energy Togo', 'Global New Energy Togo') + '_' + data['name']
    data['effective_date'] = ""
    data['start_date'] = datetime.strptime(key, '%Y.%m').strftime('%Y-%m-%d')
    data['end_date'] = ""
    data['value'] = data['value'].map(lambda x:round(x) if not (pd.isna(x) or isinstance(x, str)) else float('nan'))
    data['table'] = 'spot'
    data = data[["name","effective_date","start_date","end_date","value","table"]]
    
    return data

   
def main():
#     download_file()
#     time.sleep(3)
    translate_file(DATAFILES_DIR + 'owner_full_raw.xlsx')


if __name__ == '__main__':
    main()
